In [66]:
import matplotlib.pyplot as plt

from keras.applications import MobileNet

In [51]:
img_row,img_col= 224,224

mob=MobileNet(input_shape=(img_row,img_col,3),
              include_top=False,
              weights='imagenet')

In [52]:
for layer in mob.layers:
    layer.trainable=False

for (i,layer) in enumerate(mob.layers):
    print(str(i) +  '  ' + layer.__class__.__name__  , layer.trainable)
    

0  InputLayer False
1  ZeroPadding2D False
2  Conv2D False
3  BatchNormalization False
4  ReLU False
5  DepthwiseConv2D False
6  BatchNormalization False
7  ReLU False
8  Conv2D False
9  BatchNormalization False
10  ReLU False
11  ZeroPadding2D False
12  DepthwiseConv2D False
13  BatchNormalization False
14  ReLU False
15  Conv2D False
16  BatchNormalization False
17  ReLU False
18  DepthwiseConv2D False
19  BatchNormalization False
20  ReLU False
21  Conv2D False
22  BatchNormalization False
23  ReLU False
24  ZeroPadding2D False
25  DepthwiseConv2D False
26  BatchNormalization False
27  ReLU False
28  Conv2D False
29  BatchNormalization False
30  ReLU False
31  DepthwiseConv2D False
32  BatchNormalization False
33  ReLU False
34  Conv2D False
35  BatchNormalization False
36  ReLU False
37  ZeroPadding2D False
38  DepthwiseConv2D False
39  BatchNormalization False
40  ReLU False
41  Conv2D False
42  BatchNormalization False
43  ReLU False
44  DepthwiseConv2D False
45  BatchNormalizati

In [53]:
from keras.layers import Flatten,Dense,GlobalAveragePooling2D

x=mob.output
x=GlobalAveragePooling2D()(x)
x=Flatten()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(1024,activation='relu')(x)
x=Dense(512,activation='relu')(x)
x=Dense(10,activation='softmax')(x)

from keras.models import Model

model=Model(inputs= mob.input , outputs =x)
print(model.summary())

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [54]:
train_path = 'D:/imp_programm/monkey_breed_classifier/monkey_breed/train/'
test_path = 'D:/imp_programm/monkey_breed_classifier/monkey_breed/validation/'

In [123]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(zoom_range=0.2,horizontal_flip=True,width_shift_range=0.2,height_shift_range=0.2,
                                  rescale=1./255,fill_mode='nearest')
test_datgen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path,
                                               target_size=(img_row,img_col),
                                                   batch_size=32,
                                                   class_mode='categorical')                                                   

test_generator = test_datgen.flow_from_directory(test_path,
                                               target_size=(img_row,img_col),
                                                   batch_size=32,
                                                   class_mode='categorical')

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [124]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.optimizers import RMSprop

stopping = EarlyStopping( monitor='val_loss',
                         patience=3,
                         min_delta=0,
                         verbose=1,
                         restore_best_weights=True )

checkpoint = ModelCheckpoint('D:/imp_programm/monkey_breed_classifier/MONKEY_BREED_MOBILENET_MODEL.h5',
                             monitor='val_loss',
                            save_best_only=True,
                            mode='min',
                            verbose=1)

callback=[checkpoint,stopping]

In [125]:
model.compile(optimizer=RMSprop(lr=0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [126]:
history=model.fit_generator(train_generator,steps_per_epoch=1098//32,
                           epochs=10,callbacks=callback,validation_data=test_generator,validation_steps=272//32)

Epoch 1/10
34/34 [==============================] - ETA: 0s - loss: 1.0069 - accuracy: 0.9409
Epoch 00001: val_loss improved from inf to 0.20923, saving model to D:/imp_programm/monkey_breed_classifier\MONKEY_BREED_MOBILENET_MODEL.h5
34/34 [==============================] - 85s 3s/step - loss: 1.0069 - accuracy: 0.9409 - val_loss: 0.2092 - val_accuracy: 0.9570
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 0.1658 - accuracy: 0.9597
Epoch 00002: val_loss improved from 0.20923 to 0.07715, saving model to D:/imp_programm/monkey_breed_classifier\MONKEY_BREED_MOBILENET_MODEL.h5
34/34 [==============================] - 78s 2s/step - loss: 0.1658 - accuracy: 0.9597 - val_loss: 0.0772 - val_accuracy: 0.9805
Epoch 3/10
34/34 [==============================] - ETA: 0s - loss: 0.1918 - accuracy: 0.9550
Epoch 00003: val_loss did not improve from 0.07715
34/34 [==============================] - 130s 4s/step - loss: 0.1918 - accuracy: 0.9550 - val_loss: 0.1399 - val_accuracy: 0.

In [146]:
from keras.models import load_model

mod=load_model('D:/imp_programm/monkey_breed_classifier/MONKEY_BREED_MOBILENET_MODEL.h5')

In [267]:
import cv2 as cv
from os import listdir
from os.path import join,isfile
import re
from keras.preprocessing.image import load_img,img_to_array
import os

dic=  {'n0' :   'mantled_howler',          
        'n1' : 'patas_monkey'                  , 
        'n2'  : 'bald_uakari'                   , 
        'n3' :  'japanese_macaque'              , 
        'n4'  :  'pygmy_marmoset'                , 
        'n5'  :  'white_headed_capuchin'         , 
        'n6'  :  'silvery_marmoset'              , 
        'n7'  :  'common_squirrel_monkey'        , 
        'n8'  :  'black_headed_night_monkey'     , 
       'n9'  :  'nilgiri_langur' }

def drawTest(name,pred,im,true):
    monkey= dic['n'+str(pred[0])]
    black=[0,0,0]
    expanded_image=cv.copyMakeBorder(im,160,0,0,300,cv.BORDER_CONSTANT,value=black)
    cv.putText(expanded_image,'predicted -> '+monkey, (20,60), cv.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv.putText(expanded_image,'True -> '+dic[true], (60,100), cv.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv.imshow(name,expanded_image)

def getRandomImage(path):
    folder=list(filter(lambda x: os.path.isdir(os.path.join(path,x)), os.listdir(path)))
    random_dir= np.random.randint(0,len(folder))
    path_class= folder[random_dir]
    file_path= path+path_class
    file_names= [f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index= np.random.randint(0,len(file_names))
    image_name= file_names[random_file_index]
    return cv.imread(file_path +'/' + image_name),path_class

for i in range(0,10):
    inp,true=getRandomImage('D:/imp_programm/monkey_breed_classifier/monkey_breed/validation/')
    imp_o=inp.copy()
    imp_o=cv.resize(imp_o,None,fx=0.5,fy=0.5,interpolation=cv.INTER_LINEAR)
    
    inp=cv.resize(inp,(224,224),interpolation=cv.INTER_LINEAR)
    inp=inp/255
    inp=inp.reshape(1,224,224,3)
    
    red=np.argmax(mod.predict(inp,1,verbose=0),axis=1)
    drawTest('prediction',red,imp_o,true)
    cv.waitKey(0)

cv.destroyAllWindows()    

In [262]:
w='D:/imp_programm/monkey_breed_classifier/monkey_breed/validation/n3/n300.jpg'
inp=cv.imread(w)
inp=cv.resize(inp,(224,224),interpolation=cv.INTER_LINEAR)
inp=inp/255
inp=inp.reshape(1,224,224,3)
pred=np.argmax(mod.predict(inp))

name= dic['n'+str(pred)]

print('n'+ str(pred) +'  : ', dic['n'+str(pred)]  )

n3  :  japanese_macaque
